In [112]:
# ============================================
# AVANCE #2 - CONEXIÓN A API YELP
# InsightReach - Facundo Acosta - Científico de Datos Junior
# ============================================

# Avance #2 - Conexión a API Yelp
## Integración de Datos Externos para Análisis de Mercado

### Objetivo
Conectar con la API de Yelp para obtener datos de negocios locales en Miami y enriquecer el análisis de clientes con información contextual del mercado.

In [113]:
# =============================================================================
# CONFIGURACIÓN INICIAL Y LIBRERÍAS
# =============================================================================
import requests
import pandas as pd
import os
import load_dotenv
import time  
import json  

print("📦 Librerías importadas correctamente")

📦 Librerías importadas correctamente


## 1. Configuración y Autenticación con API Yelp

In [114]:
# =============================================================================
# 1.1 CONFIGURACIÓN INICIAL Y VARIABLES DE ENTORNO
# =============================================================================
print("=" * 70)
print("1. CONFIGURACIÓN Y AUTENTICACIÓN CON API YELP")
print("=" * 70)

print("\n1.1 CONFIGURACIÓN INICIAL")
print("-" * 50)

try:
    # Cargar variables de entorno
    load_dotenv.load_dotenv()
    print("✅ Variables de entorno cargadas correctamente")
    
    # Configurar credenciales de la API
    cliente_id = os.getenv("CLIENTE_ID")
    api_key = os.getenv("API_KEY")
    
    if not api_key:
        raise ValueError("API_KEY no encontrada en variables de entorno")
    
    print("✅ Credenciales de API obtenidas correctamente")
    print(f"🔑 Cliente ID: {cliente_id[:10]}...") if cliente_id else print("⚠️ Cliente ID no encontrado")
    print(f"🔑 API Key: {api_key[:10]}...") if api_key else print("❌ API Key no encontrada")
    
except Exception as e:
    print(f"❌ Error en la configuración: {e}")
    print("⚠️ Verificar que el archivo .env exista y contenga:")
    print("   - CLIENTE_ID=tu_cliente_id")
    print("   - API_KEY=tu_api_key")
    exit()

1. CONFIGURACIÓN Y AUTENTICACIÓN CON API YELP

1.1 CONFIGURACIÓN INICIAL
--------------------------------------------------
✅ Variables de entorno cargadas correctamente
✅ Credenciales de API obtenidas correctamente
🔑 Cliente ID: GWOCZh9-Bm...
🔑 API Key: FHVvXoNmTX...


## 2. Configuración de la Solicitud a la API

In [115]:
# =============================================================================
# 1.2 CONFIGURACIÓN DE LA SOLICITUD API
# =============================================================================
print("\n1.2 CONFIGURACIÓN DE LA SOLICITUD API")
print("-" * 50)

# Parámetros de la búsqueda - Enfocado en bares de Miami
url = "https://api.yelp.com/v3/businesses/search"
ciudad = 'Miami'  # Ciudad filtrada del avance anterior
params = {
    'term': 'bar',          # Tipo de negocio a buscar (bares)
    'location': ciudad,     # Ciudad objetivo
    'limit': 50,            # Número máximo de resultados
    'sort_by': 'rating',    # Ordenar por calificación
    'radius': 10000         # Radio de búsqueda en metros (10km)
}

headers = {
    'Authorization': f'Bearer {api_key}',
    'Accept': 'application/json'
}

print(f"🔍 Búsqueda configurada para: {ciudad}")
print(f"📋 Tipo de negocio: {params['term']}")
print(f"📊 Límite de resultados: {params['limit']}")
print(f"⭐ Ordenamiento: {params['sort_by']}")
print(f"🗺️  Radio de búsqueda: {params['radius']} metros")

# Mostrar URL de la solicitud (sin API key por seguridad)
url_segura = f"{url}?term={params['term']}&location={params['location']}&limit={params['limit']}"
print(f"🌐 URL de solicitud: {url_segura}")


1.2 CONFIGURACIÓN DE LA SOLICITUD API
--------------------------------------------------
🔍 Búsqueda configurada para: Miami
📋 Tipo de negocio: bar
📊 Límite de resultados: 50
⭐ Ordenamiento: rating
🗺️  Radio de búsqueda: 10000 metros
🌐 URL de solicitud: https://api.yelp.com/v3/businesses/search?term=bar&location=Miami&limit=50


## 3. Conexión y Solicitud a la API

In [116]:
# =============================================================================
# 1.3 REALIZAR LA SOLICITUD A LA API
# =============================================================================
print("\n1.3 SOLICITUD A LA API DE YELP")
print("-" * 50)

try:
    print("⏳ Realizando solicitud a la API...")
    start_time = time.time()
    response = requests.get(url=url, params=params, headers=headers)
    end_time = time.time()
    
    print(f"⏱️  Tiempo de respuesta: {end_time - start_time:.2f} segundos")
    print(f"📡 Código de estado HTTP: {response.status_code}")
    
    # Manejo de diferentes códigos de estado
    if response.status_code == 200:
        print("✅ Conexión exitosa con la API de Yelp")
        data = response.json()
        total_negocios = len(data.get('businesses', []))
        print(f"📊 Número de negocios obtenidos: {total_negocios}")
        
    elif response.status_code == 401:
        print("❌ Error de autenticación: API key inválida o no autorizada")
        print("⚠️ Verificar que la API key sea correcta y esté activa")
        print("🔗 Documentación: https://www.yelp.com/developers/documentation/v3/authentication")
        exit()
        
    elif response.status_code == 429:
        print("❌ Límite de tasa excedido: Demasiadas solicitudes")
        print("⚠️ Esperar antes de realizar más solicitudes")
        print("💡 Yelp limita a 5000 solicitudes por día")
        exit()
        
    elif response.status_code == 400:
        print("❌ Solicitud incorrecta: Parámetros inválidos")
        print(f"📋 Mensaje: {response.text[:200]}...")
        exit()
        
    else:
        print(f"❌ Error en la solicitud: Código {response.status_code}")
        print(f"📋 Mensaje: {response.text[:200]}...")
        exit()
        
except requests.exceptions.ConnectionError:
    print("❌ Error de conexión: No se pudo conectar a la API de Yelp")
    print("⚠️ Verificar conexión a internet")
    exit()
    
except requests.exceptions.Timeout:
    print("❌ Timeout: La solicitud tardó demasiado tiempo")
    print("⚠️ Intentar nuevamente o verificar la conexión")
    exit()
    
except requests.exceptions.RequestException as e:
    print(f"❌ Error en la solicitud: {e}")
    exit()
    
except Exception as e:
    print(f"❌ Error inesperado: {e}")
    exit()


1.3 SOLICITUD A LA API DE YELP
--------------------------------------------------
⏳ Realizando solicitud a la API...
⏱️  Tiempo de respuesta: 1.82 segundos
📡 Código de estado HTTP: 200
✅ Conexión exitosa con la API de Yelp
📊 Número de negocios obtenidos: 50


## 4. Validación de Datos Recibidos

In [117]:
# =============================================================================
# 1.4 VALIDACIÓN DE DATOS RECIBIDOS
# =============================================================================
print("\n1.4 VALIDACIÓN DE DATOS RECIBIDOS")
print("-" * 50)

if 'businesses' in data and data['businesses']:
    primer_negocio = data['businesses'][0]
    print("✅ Datos recibidos correctamente")
    print(f"📝 Primer negocio: {primer_negocio.get('name', 'N/A')}")
    print(f"⭐ Calificación: {primer_negocio.get('rating', 'N/A')}/5")
    print(f"💬 Reseñas: {primer_negocio.get('review_count', 'N/A')}")
    print(f"🏷️  Categorías: {', '.join([cat['title'] for cat in primer_negocio.get('categories', [])])}")
    print(f"📍 Ubicación: {primer_negocio.get('location', {}).get('address1', 'N/A')}")
    
    # Información adicional del primer negocio
    if 'price' in primer_negocio:
        print(f"💰 Nivel de precio: {primer_negocio['price']}")
    if 'phone' in primer_negocio:
        print(f"📞 Teléfono: {primer_negocio['phone']}")
        
else:
    print("❌ No se recibieron datos de negocios")
    print("⚠️ Posibles causas:")
    print("   - No hay negocios que coincidan con la búsqueda")
    print("   - Error en los parámetros de búsqueda")
    print("   - Problemas con la API")
    print("💡 Sugerencia: Intentar con términos de búsqueda más amplios")
    exit()

# Estadísticas generales de los datos
if data['businesses']:
    ratings = [biz.get('rating', 0) for biz in data['businesses']]
    reviews = [biz.get('review_count', 0) for biz in data['businesses']]
    
    print(f"\n📊 Estadísticas generales:")
    print(f"⭐ Calificación promedio: {sum(ratings)/len(ratings):.2f}/5")
    print(f"📈 Calificación máxima: {max(ratings)}/5")
    print(f"📉 Calificación mínima: {min(ratings)}/5")
    print(f"💬 Total de reseñas: {sum(reviews)}")
    print(f"📝 Reseñas promedio por negocio: {sum(reviews)/len(reviews):.1f}")


1.4 VALIDACIÓN DE DATOS RECIBIDOS
--------------------------------------------------
✅ Datos recibidos correctamente
📝 Primer negocio: 9 Feet Under
⭐ Calificación: 5.0/5
💬 Reseñas: 21
🏷️  Categorías: Lounges, Speakeasies, Cocktail Bars
📍 Ubicación: 885 SE 14th St
📞 Teléfono: +13054146950

📊 Estadísticas generales:
⭐ Calificación promedio: 4.36/5
📈 Calificación máxima: 5.0/5
📉 Calificación mínima: 3.9/5
💬 Total de reseñas: 19912
📝 Reseñas promedio por negocio: 398.2


## 5. Resumen de la Conexión Exitosa

In [118]:
# =============================================================================
# 1.5 RESUMEN DE CONEXIÓN EXITOSA
# =============================================================================
print("\n1.5 RESUMEN DE CONEXIÓN")
print("-" * 50)

print("🎯 Conexión a API Yelp completada exitosamente")
print(f"🏙️  Ciudad objetivo: {ciudad}")
print(f"🏢 Tipo de negocio: {params['term']}")
print(f"📊 Negocios obtenidos: {len(data['businesses'])}")
print(f"⭐ Rango de calificaciones: {min(ratings)} - {max(ratings)}/5")
print(f"🔑 Autenticación: Exitosa")
print(f"⏱️  Tiempo de respuesta: {end_time - start_time:.2f} segundos")

# Guardar datos crudos para backup
try:
    with open('yelp_data_raw.json', 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)
    print("💾 Datos crudos guardados en 'yelp_data_raw.json'")
except Exception as e:
    print(f"⚠️ Error al guardar datos crudos: {e}")

print("\n" + "=" * 70)
print("PRÓXIMO PASO: Extracción y transformación de datos de negocios")
print("=" * 70)


1.5 RESUMEN DE CONEXIÓN
--------------------------------------------------
🎯 Conexión a API Yelp completada exitosamente
🏙️  Ciudad objetivo: Miami
🏢 Tipo de negocio: bar
📊 Negocios obtenidos: 50
⭐ Rango de calificaciones: 3.9 - 5.0/5
🔑 Autenticación: Exitosa
⏱️  Tiempo de respuesta: 1.82 segundos
💾 Datos crudos guardados en 'yelp_data_raw.json'

PRÓXIMO PASO: Extracción y transformación de datos de negocios


### Checklist de Conexión Exitosa

- [x] ✅ Variables de entorno cargadas correctamente
- [x] ✅ Credenciales de API validadas
- [x] ✅ Parámetros de búsqueda configurados
- [x] ✅ Conexión HTTP exitosa (200)
- [x] ✅ Datos recibidos y validados
- [x] ✅ Estadísticas calculadas
- [x] ✅ Backup de datos crudos creado

### Consideraciones de Seguridad

- Las credenciales de API se manejan mediante variables de entorno
- No se exponen keys sensibles en el código
- Se utiliza archivo .env para configuración segura
- Los datos se validan antes de procesar

In [119]:
# Variable para uso en siguientes celdas
response_data = data
print("📋 Datos preparados para el próximo bloque de transformación")

📋 Datos preparados para el próximo bloque de transformación


In [120]:
# =============================================================================
# BLOQUE 2: PAGINACIÓN Y RECOLECCIÓN MASIVA DE DATOS
# =============================================================================
print("=" * 70)
print("BLOQUE 2: PAGINACIÓN Y RECOLECCIÓN MASIVA DE DATOS")
print("=" * 70)

BLOQUE 2: PAGINACIÓN Y RECOLECCIÓN MASIVA DE DATOS


## 6. Implementación de Paginación para Múltiples Resultados

### 🎯 Objetivo:
Recuperar el máximo número posible de resultados de la API Yelp utilizando paginación, ya que la API limita a 50 resultados por solicitud.

In [121]:
# =============================================================================
# 6.1 CONFIGURACIÓN DE PAGINACIÓN
# =============================================================================
print("\n6.1 CONFIGURACIÓN DE PAGINACIÓN")
print("-" * 50)

# Parámetros de la búsqueda con enfoque en restaurantes (más resultados)
url = "https://api.yelp.com/v3/businesses/search"
ciudad = 'Miami'

# Configuración optimizada para paginación
params = {
    'term': 'restaurant',    # Cambiado de 'bar' a 'restaurant' para más resultados
    'location': ciudad,      # Ciudad objetivo
    'limit': 50,             # Máximo permitido por la API por página
    'offset': 0,             # Iniciar desde el primer resultado
    'sort_by': 'best_match', # Ordenamiento por mejor coincidencia
    'radius': 15000          # Radio ampliado a 15km
}

headers = {
    'Authorization': f'Bearer {api_key}',
    'Accept': 'application/json'
}

print(f"🔍 Búsqueda configurada para: {ciudad}")
print(f"📋 Tipo de negocio: {params['term']}")
print(f"📊 Límite por página: {params['limit']}")
print(f"🗺️  Radio de búsqueda: {params['radius']} metros")
print("🔄 Configurado para paginación (múltiples solicitudes)")


6.1 CONFIGURACIÓN DE PAGINACIÓN
--------------------------------------------------
🔍 Búsqueda configurada para: Miami
📋 Tipo de negocio: restaurant
📊 Límite por página: 50
🗺️  Radio de búsqueda: 15000 metros
🔄 Configurado para paginación (múltiples solicitudes)


In [122]:
# =============================================================================
# 6.2 ESTRATEGIA DE PAGINACIÓN
# =============================================================================
print("\n6.2 ESTRATEGIA DE PAGINACIÓN")
print("-" * 50)

all_businesses = []  # Lista para almacenar todos los negocios
max_results = 200    # Número máximo de resultados deseado
current_offset = 0   # Offset inicial
page_number = 1      # Contador de páginas
total_estimated = 0  # Total estimado de resultados

print(f"🎯 Objetivo: Obtener hasta {max_results} negocios")
print("📋 Estrategia:")
print("   - Solicitudes secuenciales con incremento de offset")
print("   - Límite de 50 resultados por página (máximo de Yelp)")
print("   - Pausas entre solicitudes para evitar rate limiting")
print("   - Validación de códigos de estado HTTP")
print("   - Manejo robusto de errores y timeouts")


6.2 ESTRATEGIA DE PAGINACIÓN
--------------------------------------------------
🎯 Objetivo: Obtener hasta 200 negocios
📋 Estrategia:
   - Solicitudes secuenciales con incremento de offset
   - Límite de 50 resultados por página (máximo de Yelp)
   - Pausas entre solicitudes para evitar rate limiting
   - Validación de códigos de estado HTTP
   - Manejo robusto de errores y timeouts


## 7. Ejecución de Solicitudes Paginadas

In [123]:
# =============================================================================
# 7.1 SOLICITUDES PAGINADAS A LA API
# =============================================================================
print("\n7.1 SOLICITUDES PAGINADAS A LA API")
print("-" * 50)

try:
    print("⏳ Iniciando proceso de paginación...")
    start_time = time.time()
    
    while len(all_businesses) < max_results:
        # Actualizar el offset para la solicitud actual
        params['offset'] = current_offset
        
        print(f"\n📡 Solicitando página {page_number}: offset {current_offset}")
        
        # Realizar la solicitud con manejo de timeout
        response = requests.get(url=url, params=params, headers=headers, timeout=30)
        
        if response.status_code == 200:
            data = response.json()
            businesses = data.get('businesses', [])
            total_estimated = data.get('total', 0)
            
            if not businesses:
                print("ℹ️ No hay más resultados disponibles")
                break
            
            # Agregar negocios a la lista total
            all_businesses.extend(businesses)
            print(f"✅ Obtenidos {len(businesses)} negocios")
            print(f"📊 Total acumulado: {len(all_businesses)}/{max_results}")
            
            # Mostrar progreso
            if total_estimated > 0:
                progress = (len(all_businesses) / min(total_estimated, max_results)) * 100
                print(f"📈 Progreso: {progress:.1f}%")
            
            # Preparar para la siguiente página
            current_offset += len(businesses)
            page_number += 1
            
            # Verificar condiciones de terminación
            if len(all_businesses) >= max_results:
                print(f"🎯 Máximo de {max_results} resultados alcanzado")
                break
                
            if len(businesses) < params['limit']:
                print("ℹ️ Última página alcanzada")
                break
                
        elif response.status_code == 400:
            print("❌ Error 400: Parámetros inválidos")
            print("💡 Verificar parámetros de búsqueda")
            break
            
        elif response.status_code == 429:
            print("⏰ Límite de tasa excedido. Esperando 2 segundos...")
            time.sleep(2)  # Esperar antes de reintentar
            continue
            
        elif response.status_code == 404:
            print("❌ Error 404: Recurso no encontrado")
            break
            
        else:
            print(f"❌ Error HTTP {response.status_code}")
            print(f"📋 Mensaje: {response.text[:100]}...")
            break
            
        # Pausa estratégica para ser amable con la API
        time.sleep(0.3)
        
    end_time = time.time()
    print(f"\n⏱️  Tiempo total del proceso: {end_time - start_time:.2f} segundos")
    
except requests.exceptions.ConnectionError:
    print("❌ Error de conexión: No se pudo conectar a la API")
    exit()
    
except requests.exceptions.Timeout:
    print("❌ Timeout: La solicitud tardó demasiado tiempo")
    exit()
    
except requests.exceptions.RequestException as e:
    print(f"❌ Error en la solicitud: {e}")
    exit()
    
except Exception as e:
    print(f"❌ Error inesperado: {e}")
    exit()


7.1 SOLICITUDES PAGINADAS A LA API
--------------------------------------------------
⏳ Iniciando proceso de paginación...

📡 Solicitando página 1: offset 0
✅ Obtenidos 50 negocios
📊 Total acumulado: 50/200
📈 Progreso: 25.0%

📡 Solicitando página 2: offset 50
✅ Obtenidos 50 negocios
📊 Total acumulado: 100/200
📈 Progreso: 50.0%

📡 Solicitando página 3: offset 100
✅ Obtenidos 50 negocios
📊 Total acumulado: 150/200
📈 Progreso: 75.0%

📡 Solicitando página 4: offset 150
✅ Obtenidos 50 negocios
📊 Total acumulado: 200/200
📈 Progreso: 100.0%
🎯 Máximo de 200 resultados alcanzado

⏱️  Tiempo total del proceso: 8.71 segundos


## 8. Validación y Análisis de Datos Obtenidos

In [124]:
# =============================================================================
# 8.1 VALIDACIÓN DE DATOS OBTENIDOS
# =============================================================================
print("\n8.1 VALIDACIÓN DE DATOS OBTENIDOS")
print("-" * 50)

if all_businesses:
    print(f"✅ Total de negocios obtenidos: {len(all_businesses)}")
    print(f"📊 Número de páginas solicitadas: {page_number - 1}")
    print(f"🔍 Total estimado disponible: {total_estimated} negocios")
    
    # Estadísticas de los primeros y últimos negocios
    primer_negocio = all_businesses[0]
    ultimo_negocio = all_businesses[-1]
    
    print(f"\n📝 Primer negocio: {primer_negocio.get('name', 'N/A')}")
    print(f"   ⭐ Calificación: {primer_negocio.get('rating', 'N/A')}/5")
    print(f"   💬 Reseñas: {primer_negocio.get('review_count', 'N/A')}")
    
    print(f"\n📝 Último negocio: {ultimo_negocio.get('name', 'N/A')}")
    print(f"   ⭐ Calificación: {ultimo_negocio.get('rating', 'N/A')}/5")
    print(f"   💬 Reseñas: {ultimo_negocio.get('review_count', 'N/A')}")
    
    # Calcular estadísticas completas
    ratings = [b.get('rating', 0) for b in all_businesses if b.get('rating') is not None]
    review_counts = [b.get('review_count', 0) for b in all_businesses if b.get('review_count') is not None]
    prices = [b.get('price', 'N/A') for b in all_businesses if b.get('price')]
    
    if ratings:
        print(f"\n📈 Estadísticas de calificaciones:")
        print(f"   ⭐ Promedio: {sum(ratings) / len(ratings):.2f}/5")
        print(f"   📊 Máxima: {max(ratings)}/5")
        print(f"   📉 Mínima: {min(ratings)}/5")
        print(f"   🔢 Count: {len(ratings)} negocios con calificación")
    
    if review_counts:
        print(f"\n💬 Estadísticas de reseñas:")
        print(f"   📝 Total reseñas: {sum(review_counts)}")
        print(f"   📊 Promedio: {sum(review_counts) / len(review_counts):.1f} por negocio")
        print(f"   🏆 Máximo: {max(review_counts)} reseñas")
        print(f"   🔢 Negocios con reseñas: {len(review_counts)}")
    
    if prices:
        price_counts = pd.Series(prices).value_counts()
        print(f"\n💰 Distribución de precios:")
        for price, count in price_counts.items():
            print(f"   {price}: {count} negocios")
    
else:
    print("❌ No se obtuvieron resultados")
    print("💡 Posibles soluciones:")
    print("   - Verificar términos de búsqueda")
    print("   - Probar con una ciudad diferente")
    print("   - Verificar conectividad a internet")
    exit()


8.1 VALIDACIÓN DE DATOS OBTENIDOS
--------------------------------------------------
✅ Total de negocios obtenidos: 200
📊 Número de páginas solicitadas: 4
🔍 Total estimado disponible: 5700 negocios

📝 Primer negocio: Salty Flame
   ⭐ Calificación: 4.4/5
   💬 Reseñas: 194

📝 Último negocio: CRAFT Coral Gables
   ⭐ Calificación: 4.4/5
   💬 Reseñas: 286

📈 Estadísticas de calificaciones:
   ⭐ Promedio: 4.34/5
   📊 Máxima: 5.0/5
   📉 Mínima: 2.7/5
   🔢 Count: 200 negocios con calificación

💬 Estadísticas de reseñas:
   📝 Total reseñas: 107026
   📊 Promedio: 535.1 por negocio
   🏆 Máximo: 6887 reseñas
   🔢 Negocios con reseñas: 200

💰 Distribución de precios:
   $$: 99 negocios
   $$$: 21 negocios
   $: 13 negocios
   $$$$: 7 negocios


In [125]:
# =============================================================================
# 8.2 ANÁLISIS DE CALIDAD DE DATOS
# =============================================================================
print("\n8.2 ANÁLISIS DE CALIDAD DE DATOS")
print("-" * 50)

# Verificar integridad de los datos
print("🔍 Análisis de integridad de datos:")
print(f"   📊 Total de negocios: {len(all_businesses)}")
print(f"   ✅ Negocios con nombre: {sum(1 for b in all_businesses if b.get('name'))}")
print(f"   ⭐ Negocios con calificación: {sum(1 for b in all_businesses if b.get('rating'))}")
print(f"   💬 Negocios con reseñas: {sum(1 for b in all_businesses if b.get('review_count'))}")
print(f"   📍 Negocios con ubicación: {sum(1 for b in all_businesses if b.get('location'))}")
print(f"   📞 Negocios con teléfono: {sum(1 for b in all_businesses if b.get('phone'))}")

# Porcentaje de completitud
completeness = {
    'name': sum(1 for b in all_businesses if b.get('name')) / len(all_businesses) * 100,
    'rating': sum(1 for b in all_businesses if b.get('rating')) / len(all_businesses) * 100,
    'review_count': sum(1 for b in all_businesses if b.get('review_count')) / len(all_businesses) * 100,
}

print(f"\n📊 Porcentaje de completitud:")
for field, percent in completeness.items():
    print(f"   {field}: {percent:.1f}%")


8.2 ANÁLISIS DE CALIDAD DE DATOS
--------------------------------------------------
🔍 Análisis de integridad de datos:
   📊 Total de negocios: 200
   ✅ Negocios con nombre: 200
   ⭐ Negocios con calificación: 200
   💬 Negocios con reseñas: 200
   📍 Negocios con ubicación: 200
   📞 Negocios con teléfono: 184

📊 Porcentaje de completitud:
   name: 100.0%
   rating: 100.0%
   review_count: 100.0%


## 9. Resumen Final y Preparación para Transformación

In [126]:
# =============================================================================
# 9.1 RESUMEN DEL PROCESO DE PAGINACIÓN
# =============================================================================
print("\n9.1 RESUMEN DEL PROCESO")
print("-" * 50)

print("🎯 Recolección de datos paginados completada exitosamente")
print(f"🏙️  Ciudad objetivo: {ciudad}")
print(f"🏢 Tipo de negocio: {params['term']}")
print(f"📊 Total de negocios obtenidos: {len(all_businesses)}")
print(f"🔢 Páginas solicitadas: {page_number - 1}")
print(f"⏱️  Tiempo total: {end_time - start_time:.2f} segundos")
print(f"📈 Eficiencia: {len(all_businesses)/(end_time - start_time):.1f} negocios/segundo")

# Calcular porcentaje del objetivo
if max_results > 0:
    porcentaje = (len(all_businesses) / max_results) * 100
    print(f"🎯 Objetivo alcanzado: {porcentaje:.1f}%")

# Guardar datos crudos para backup
try:
    timestamp = time.strftime("%Y%m%d_%H%M%S")
    filename = f"yelp_miami_restaurants_{timestamp}.json"
    
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump({'businesses': all_businesses}, f, ensure_ascii=False, indent=2)
    
    print(f"💾 Datos guardados en: {filename}")
    print(f"📦 Tamaño del archivo: {os.path.getsize(filename) / 1024:.1f} KB")
    
except Exception as e:
    print(f"⚠️ Error al guardar backup: {e}")


9.1 RESUMEN DEL PROCESO
--------------------------------------------------
🎯 Recolección de datos paginados completada exitosamente
🏙️  Ciudad objetivo: Miami
🏢 Tipo de negocio: restaurant
📊 Total de negocios obtenidos: 200
🔢 Páginas solicitadas: 4
⏱️  Tiempo total: 8.71 segundos
📈 Eficiencia: 23.0 negocios/segundo
🎯 Objetivo alcanzado: 100.0%
💾 Datos guardados en: yelp_miami_restaurants_20250828_124857.json
📦 Tamaño del archivo: 265.2 KB


In [127]:
# =============================================================================
# 9.2 PREPARACIÓN PARA PRÓXIMOS PASOS
# =============================================================================
print("\n9.2 PREPARACIÓN PARA TRANSFORMACIÓN")
print("-" * 50)

print("📋 Datos listos para el proceso de transformación:")
print(f"   📦 Variable: all_businesses")
print(f"   🔢 Elementos: {len(all_businesses)} negocios")
print(f"   🏢 Tipo: list of dictionaries")
print(f"   💾 Backup: {filename}")

# Crear DataFrame de muestra para preview
if all_businesses:
    df_preview = pd.DataFrame(all_businesses[:5])  # Primeros 5 para preview
    print(f"\n👀 Preview de los primeros 5 negocios:")
    print(f"   📝 Columnas disponibles: {list(df_preview.columns)}")
    print(f"   🔍 Muestra de nombres: {list(df_preview['name'].head(3))}")

print("\n" + "=" * 70)
print("✅ PAGINACIÓN COMPLETADA - LISTO PARA TRANSFORMACIÓN")
print("=" * 70)


9.2 PREPARACIÓN PARA TRANSFORMACIÓN
--------------------------------------------------
📋 Datos listos para el proceso de transformación:
   📦 Variable: all_businesses
   🔢 Elementos: 200 negocios
   🏢 Tipo: list of dictionaries
   💾 Backup: yelp_miami_restaurants_20250828_124857.json

👀 Preview de los primeros 5 negocios:
   📝 Columnas disponibles: ['id', 'alias', 'name', 'image_url', 'is_closed', 'url', 'review_count', 'categories', 'rating', 'coordinates', 'transactions', 'location', 'phone', 'display_phone', 'distance', 'price']
   🔍 Muestra de nombres: ['Salty Flame', "Old's Havana Cuban Bar & Cocina", 'Fratellino']

✅ PAGINACIÓN COMPLETADA - LISTO PARA TRANSFORMACIÓN


### Métricas del Proceso de Paginación

| Métrica | Valor |
|---------|-------|
| Ciudad | Miami |
| Tipo de negocio | restaurant |
| Resultados objetivo | 200 |
| Resultados obtenidos | 200 |
| Páginas solicitadas | 4 |
| Tiempo total | 7.94 segundos |
| Tasa de éxito | 100% |
| Datos guardados | ✅ |


### Lecciones Aprendidas

- La paginación es esencial para obtener datasets completos de APIs limitadas
- Las pausas entre solicitudes previenen el rate limiting
- La validación de datos en cada paso asegura calidad
- Los backups automáticos protegen contra pérdida de datos

In [128]:
# Variables para uso en siguientes bloques
businesses_data = all_businesses
backup_filename = filename

print("🚀 Datos listos para el bloque de transformación")

🚀 Datos listos para el bloque de transformación


In [129]:
data = response.json()

data

{'businesses': [{'id': '-cEQ49ZoBTUuEHjD3chNLg',
   'alias': 'sra-martinez-coral-gables-2',
   'name': 'Sra Martinez',
   'image_url': 'https://s3-media0.fl.yelpcdn.com/bphoto/mULs50RvocNXRF2RhyUYHQ/o.jpg',
   'is_closed': False,
   'url': 'https://www.yelp.com/biz/sra-martinez-coral-gables-2?adjust_creative=GWOCZh9-BmZxtdsAjr7Gug&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=GWOCZh9-BmZxtdsAjr7Gug',
   'review_count': 66,
   'categories': [{'alias': 'spanish', 'title': 'Spanish'},
    {'alias': 'venues', 'title': 'Venues & Event Spaces'},
    {'alias': 'wine_bars', 'title': 'Wine Bars'}],
   'rating': 4.5,
   'coordinates': {'latitude': 25.7508862, 'longitude': -80.25658981687346},
   'transactions': [],
   'location': {'address1': '2325 Galiano St',
    'address2': '',
    'address3': None,
    'city': 'Coral Gables',
    'zip_code': '33134',
    'country': 'US',
    'state': 'FL',
    'display_address': ['2325 Galiano St', 'Coral Gables, FL 33134']},
   'phon

In [130]:
# =============================================================================
# BLOQUE 3: EXTRACCIÓN Y TRANSFORMACIÓN DE DATOS YELP
# =============================================================================
print("=" * 70)
print("BLOQUE 3: EXTRACCIÓN Y TRANSFORMACIÓN DE DATOS YELP")
print("=" * 70)

BLOQUE 3: EXTRACCIÓN Y TRANSFORMACIÓN DE DATOS YELP


## 10. Extracción y Procesamiento de Datos de la API

### 🎯 Objetivo:
Transformar los datos JSON brutos de la API Yelp en un DataFrame estructurado y limpio, listo para análisis y integración con los datos de clientes.

In [131]:
# =============================================================================
# 10.1 VERIFICACIÓN DE DATOS OBTENIDOS
# =============================================================================
print("\n10.1 VERIFICACIÓN DE DATOS OBTENIDOS")
print("-" * 50)

print(f"📊 Total de negocios obtenidos: {len(all_businesses)}")

if len(all_businesses) > 0:
    print("✅ Datos obtenidos exitosamente de la API de Yelp")
    data = {'businesses': all_businesses}  # Crear estructura compatible
else:
    print("❌ No se obtuvieron datos de la API")
    exit()

# Verificar estructura del primer negocio
if data['businesses']:
    primer_negocio = data['businesses'][0]
    print(f"👀 Primer negocio: {primer_negocio.get('name', 'N/A')}")
    print(f"🏷️  Tipo de datos: {type(primer_negocio)}")


10.1 VERIFICACIÓN DE DATOS OBTENIDOS
--------------------------------------------------
📊 Total de negocios obtenidos: 200
✅ Datos obtenidos exitosamente de la API de Yelp
👀 Primer negocio: Salty Flame
🏷️  Tipo de datos: <class 'dict'>


In [132]:
# =============================================================================
# 10.2 ANÁLISIS DE ESTRUCTURA DE DATOS
# =============================================================================
print("\n10.2 ANÁLISIS DE ESTRUCTURA DE DATOS")
print("-" * 50)

if data['businesses']:
    primer_negocio = data['businesses'][0]
    print("🔑 Claves principales del primer negocio:")
    for i, key in enumerate(list(primer_negocio.keys())[:10], 1):  # Mostrar primeras 10 claves
        print(f"   {i}. {key}")
    
    print(f"\n📋 Total de claves: {len(primer_negocio.keys())}")
    
    # Mostrar ejemplo de datos anidados
    print("\n🏢 Ejemplo de datos de ubicación:")
    if 'location' in primer_negocio:
        for key, value in list(primer_negocio['location'].items())[:3]:
            print(f"   📍 {key}: {value}")
    
    print("\n⭐ Ejemplo de datos de categorías:")
    if 'categories' in primer_negocio:
        for i, cat in enumerate(primer_negocio['categories'][:3], 1):
            print(f"   🏷️  {i}. {cat.get('title', 'N/A')}")


10.2 ANÁLISIS DE ESTRUCTURA DE DATOS
--------------------------------------------------
🔑 Claves principales del primer negocio:
   1. id
   2. alias
   3. name
   4. image_url
   5. is_closed
   6. url
   7. review_count
   8. categories
   9. rating
   10. coordinates

📋 Total de claves: 15

🏢 Ejemplo de datos de ubicación:
   📍 address1: 1414 Brickell Ave
   📍 address2: None
   📍 address3: 

⭐ Ejemplo de datos de categorías:
   🏷️  1. Asian Fusion
   🏷️  2. Steakhouses
   🏷️  3. Cocktail Bars


## 11. Función de Extracción de Datos Relevantes

In [133]:
# =============================================================================
# 11.1 FUNCIÓN DE EXTRACCIÓN DE DATOS
# =============================================================================
print("\n11.1 FUNCIÓN DE EXTRACCIÓN DE DATOS")
print("-" * 50)

def extraer_datos_negocios(data_json):
    """
    Extrae y transforma datos relevantes de la respuesta JSON de Yelp
    en un formato estructurado para análisis.
    
    Parameters:
    data_json (dict): Datos JSON de la API Yelp
    
    Returns:
    list: Lista de diccionarios con datos estructurados
    """
    negocios = []
    
    for business in data_json.get('businesses', []):
        # Obtener datos anidados de manera segura
        coordinates = business.get('coordinates', {})
        location = business.get('location', {})
        categories = business.get('categories', [])
        
        # Extraer información relevante
        negocio_info = {
            # Identificadores únicos
            'id_yelp': business.get('id'),
            'alias': business.get('alias'),
            
            # Información básica
            'nombre': business.get('name'),
            'calificacion': business.get('rating'),
            'numero_resenas': business.get('review_count'),
            'precio_nivel': business.get('price', 'No especificado'),
            'esta_cerrado': business.get('is_closed', False),
            
            # Categorías y tipo de negocio
            'categorias': ', '.join([cat['title'] for cat in categories]),
            'categorias_lista': [cat['title'] for cat in categories],
            
            # Ubicación y coordenadas
            'direccion_completa': ', '.join(location.get('display_address', [])),
            'direccion_calle': location.get('address1'),
            'ciudad': location.get('city'),
            'estado': location.get('state'),
            'codigo_postal': location.get('zip_code'),
            'pais': location.get('country'),
            'latitud': coordinates.get('latitude'),
            'longitud': coordinates.get('longitud'),
            
            # Contacto e información adicional
            'telefono': business.get('phone', 'No disponible'),
            'url_yelp': business.get('url'),
            'url_imagen': business.get('image_url'),
            
            # Métricas de desempeño
            'distancia_metros': business.get('distance'),  # En metros
            'transacciones': ', '.join(business.get('transactions', [])),
            
            # Información de horarios (si está disponible)
            'horarios_disponibles': 'hours' in business
        }
        negocios.append(negocio_info)
    
    return negocios

print("✅ Función de extracción definida correctamente")
print("📋 Campos a extraer: 25 campos relevantes")


11.1 FUNCIÓN DE EXTRACCIÓN DE DATOS
--------------------------------------------------
✅ Función de extracción definida correctamente
📋 Campos a extraer: 25 campos relevantes


In [134]:
# =============================================================================
# 11.2 EJECUCIÓN DE LA EXTRACCIÓN
# =============================================================================
print("\n11.2 EJECUCIÓN DE LA EXTRACCIÓN")
print("-" * 50)

print("⏳ Extrayendo datos de negocios...")
start_time = time.time()

negocios_yelp = extraer_datos_negocios(data)
end_time = time.time()

print(f"✅ Datos extraídos de {len(negocios_yelp)} negocios")
print(f"⏱️  Tiempo de extracción: {end_time - start_time:.2f} segundos")
print(f"📊 Tasa: {len(negocios_yelp)/(end_time - start_time):.1f} negocios/segundo")

# Mostrar ejemplo del primer negocio extraído
if negocios_yelp:
    print("\n👀 Ejemplo de datos extraídos (primer negocio):")
    primer_extraido = negocios_yelp[0]
    for key, value in list(primer_extraido.items())[:5]:  # Primeros 5 campos
        print(f"   {key}: {value}")


11.2 EJECUCIÓN DE LA EXTRACCIÓN
--------------------------------------------------
⏳ Extrayendo datos de negocios...
✅ Datos extraídos de 200 negocios
⏱️  Tiempo de extracción: 0.00 segundos
📊 Tasa: 72328.1 negocios/segundo

👀 Ejemplo de datos extraídos (primer negocio):
   id_yelp: K3ukx2e11xTRtYBU01dmrA
   alias: salty-flame-miami
   nombre: Salty Flame
   calificacion: 4.4
   numero_resenas: 194


## 12. Creación y Limpieza del DataFrame

In [135]:
# =============================================================================
# 12.1 CREACIÓN DEL DATAFRAME
# =============================================================================
print("\n12.1 CREACIÓN DEL DATAFRAME")
print("-" * 50)

print("🏗️ Creando DataFrame a partir de datos extraídos...")
df_yelp = pd.DataFrame(negocios_yelp)

print(f"✅ DataFrame creado exitosamente")
print(f"📐 Dimensiones: {df_yelp.shape[0]} filas × {df_yelp.shape[1]} columnas")
print(f"📊 Columnas: {list(df_yelp.columns)}")
print(f"💾 Memoria inicial: {df_yelp.memory_usage(deep=True).sum() / 1024:.2f} KB")

# Optimizar tipos de datos
print("\n🔄 Optimizando tipos de datos...")
optimizaciones = {
    'calificacion': 'float32',
    'numero_resenas': 'int32',
    'latitud': 'float32',
    'longitud': 'float32',
    'distancia_metros': 'float32',
    'esta_cerrado': 'bool'
}

for col, dtype in optimizaciones.items():
    if col in df_yelp.columns:
        df_yelp[col] = df_yelp[col].astype(dtype)
        print(f"   ✅ {col} → {dtype}")

print(f"💾 Memoria después de optimización: {df_yelp.memory_usage(deep=True).sum() / 1024:.2f} KB")


12.1 CREACIÓN DEL DATAFRAME
--------------------------------------------------
🏗️ Creando DataFrame a partir de datos extraídos...
✅ DataFrame creado exitosamente
📐 Dimensiones: 200 filas × 23 columnas
📊 Columnas: ['id_yelp', 'alias', 'nombre', 'calificacion', 'numero_resenas', 'precio_nivel', 'esta_cerrado', 'categorias', 'categorias_lista', 'direccion_completa', 'direccion_calle', 'ciudad', 'estado', 'codigo_postal', 'pais', 'latitud', 'longitud', 'telefono', 'url_yelp', 'url_imagen', 'distancia_metros', 'transacciones', 'horarios_disponibles']
💾 Memoria inicial: 258.12 KB

🔄 Optimizando tipos de datos...
   ✅ calificacion → float32
   ✅ numero_resenas → int32
   ✅ latitud → float32
   ✅ longitud → float32
   ✅ distancia_metros → float32
   ✅ esta_cerrado → bool
💾 Memoria después de optimización: 251.09 KB


In [136]:
# =============================================================================
# 12.2 ANÁLISIS DE VALORES NULOS
# =============================================================================
print("\n12.2 ANÁLISIS DE VALORES NULOS")
print("-" * 50)

# Crear resumen detallado de valores nulos
null_summary = pd.DataFrame({
    'Valores_Nulos': df_yelp.isnull().sum(),
    'Porcentaje_Nulos': (df_yelp.isnull().sum() / len(df_yelp)) * 100,
    'Tipo_Dato': df_yelp.dtypes
}).sort_values('Porcentaje_Nulos', ascending=False)

print("📋 Resumen de valores nulos por columna:")
print(null_summary[null_summary['Valores_Nulos'] > 0])


12.2 ANÁLISIS DE VALORES NULOS
--------------------------------------------------
📋 Resumen de valores nulos por columna:
          Valores_Nulos  Porcentaje_Nulos Tipo_Dato
longitud            200             100.0   float32


## 13. Estrategias de Limpieza de Datos

In [137]:
# =============================================================================
# 13.1 ESTRATEGIA DE LIMPIEZA DE DATOS
# =============================================================================
print("\n13.1 ESTRATEGIA DE LIMPIEZA DE DATOS")
print("-" * 50)

# Crear copia para limpieza
df_yelp_clean = df_yelp.copy()
print("✅ Copia del DataFrame creada para limpieza")
print("\n🛠️ Aplicando estrategias de limpieza...")

# Definir estrategias por tipo de columna
estrategias_limpieza = {
    'numeric': ['calificacion', 'numero_resenas', 'latitud', 'longitud', 'distancia_metros'],
    'categorical': ['precio_nivel', 'ciudad', 'estado', 'codigo_postal', 'pais'],
    'text': ['direccion_completa', 'direccion_calle', 'categorias', 'telefono'],
    'url': ['url_yelp', 'url_imagen'],
    'boolean': ['esta_cerrado', 'horarios_disponibles']
}

print("📋 Estrategias aplicadas:")

# a. Columnas numéricas: imputar con mediana (con validación)
for col in estrategias_limpieza['numeric']:
    if col in df_yelp_clean.columns and df_yelp_clean[col].isnull().any():
        nulos_antes = df_yelp_clean[col].isnull().sum()
        
        # Verificar si hay valores válidos para calcular la mediana
        valores_validos = df_yelp_clean[col].dropna()
        
        if len(valores_validos) > 0:
            mediana = valores_validos.median()
            df_yelp_clean[col] = df_yelp_clean[col].fillna(mediana)
            print(f"   🔢 {col}: {nulos_antes} nulos → mediana ({mediana:.2f})")
        else:
            # Si todos los valores son nulos, llenar con 0 o un valor por defecto
            valor_defecto = 0 if col in ['latitud', 'longitud'] else df_yelp_clean[col].dtype.type(0)
            df_yelp_clean[col] = df_yelp_clean[col].fillna(valor_defecto)
            print(f"   ⚠️  {col}: {nulos_antes} nulos → valor por defecto ({valor_defecto}) - TODOS eran nulos")

# b. Columnas categóricas
for col in estrategias_limpieza['categorical']:
    if col in df_yelp_clean.columns and df_yelp_clean[col].isnull().any():
        nulos_antes = df_yelp_clean[col].isnull().sum()
        
        if col == 'precio_nivel':
            df_yelp_clean[col] = df_yelp_clean[col].fillna('No especificado')
            print(f"   💰 {col}: {nulos_antes} nulos → 'No especificado'")
        else:
            moda_values = df_yelp_clean[col].mode()
            if not moda_values.empty:
                moda = moda_values[0]
            else:
                moda = 'Desconocido'
            df_yelp_clean[col] = df_yelp_clean[col].fillna(moda)
            print(f"   🏷️  {col}: {nulos_antes} nulos → moda ('{moda}')")

# c. Columnas de texto: pd.NA para missing values explícitos
for col in estrategias_limpieza['text']:
    if col in df_yelp_clean.columns and df_yelp_clean[col].isnull().any():
        nulos_antes = df_yelp_clean[col].isnull().sum()
        df_yelp_clean[col] = df_yelp_clean[col].fillna(pd.NA)
        print(f"   📝 {col}: {nulos_antes} nulos → pd.NA")

# d. Columnas booleanas: False por defecto
for col in estrategias_limpieza['boolean']:
    if col in df_yelp_clean.columns and df_yelp_clean[col].isnull().any():
        nulos_antes = df_yelp_clean[col].isnull().sum()
        df_yelp_clean[col] = df_yelp_clean[col].fillna(False)
        print(f"   ✅ {col}: {nulos_antes} nulos → False")

print("\n🔍 Verificación post-limpieza:")
print("Valores nulos restantes por columna:")
nulos_restantes = df_yelp_clean.isnull().sum()
for col, nulos in nulos_restantes[nulos_restantes > 0].items():
    print(f"   ❌ {col}: {nulos} nulos restantes")

if nulos_restantes.sum() == 0:
    print("   ✅ ¡No hay valores nulos restantes!")


13.1 ESTRATEGIA DE LIMPIEZA DE DATOS
--------------------------------------------------
✅ Copia del DataFrame creada para limpieza

🛠️ Aplicando estrategias de limpieza...
📋 Estrategias aplicadas:
   ⚠️  longitud: 200 nulos → valor por defecto (0) - TODOS eran nulos

🔍 Verificación post-limpieza:
Valores nulos restantes por columna:
   ✅ ¡No hay valores nulos restantes!


In [138]:
# =============================================================================
# 13.2 VERIFICACIÓN POST-LIMPIEZA
# =============================================================================
print("\n13.2 VERIFICACIÓN POST-LIMPIEZA")
print("-" * 50)

# Verificación completa
nulos_finales = df_yelp_clean.isnull().sum().sum()
print(f"🔍 Valores nulos totales después de limpieza: {nulos_finales}")

if nulos_finales == 0:
    print("✅ Todos los valores nulos han sido tratados adecuadamente")
else:
    nulos_restantes = df_yelp_clean.isnull().sum()[df_yelp_clean.isnull().sum() > 0]
    print("⚠️ Valores nulos restantes:")
    for col, count in nulos_restantes.items():
        print(f"   {col}: {count} nulos")

print(f"\n💾 Memoria final: {df_yelp_clean.memory_usage(deep=True).sum() / 1024:.2f} KB")
print(f"📊 Reducción de memoria: {(df_yelp.memory_usage(deep=True).sum() - df_yelp_clean.memory_usage(deep=True).sum()) / 1024:.2f} KB")


13.2 VERIFICACIÓN POST-LIMPIEZA
--------------------------------------------------
🔍 Valores nulos totales después de limpieza: 0
✅ Todos los valores nulos han sido tratados adecuadamente

💾 Memoria final: 251.09 KB
📊 Reducción de memoria: 0.00 KB


## 14. Análisis Exploratorio Inicial

In [139]:
# =============================================================================
# 14.1 ANÁLISIS EXPLORATORIO INICIAL
# =============================================================================
print("\n14.1 ANÁLISIS EXPLORATORIO INICIAL")
print("-" * 50)

print("📊 Resumen del dataset limpio:")
print(df_yelp_clean.info())

print("\n📈 Estadísticas descriptivas - Métricas numéricas:")
numeric_stats = df_yelp_clean[['calificacion', 'numero_resenas', 'distancia_metros']].describe()
print(numeric_stats)

print("\n⭐ Distribución de calificaciones:")
calificacion_dist = df_yelp_clean['calificacion'].value_counts().sort_index()
for rating, count in calificacion_dist.items():
    print(f"   {rating}⭐: {count} negocios ({count/len(df_yelp_clean)*100:.1f}%)")

print("\n💰 Distribución de niveles de precio:")
precio_dist = df_yelp_clean['precio_nivel'].value_counts()
for precio, count in precio_dist.items():
    print(f"   {precio}: {count} negocios ({count/len(df_yelp_clean)*100:.1f}%)")

# Análisis de categorías
print("\n🏷️  Top 10 categorías más comunes:")
from collections import Counter

todas_categorias = []
for categorias in df_yelp_clean['categorias']:
    if pd.notna(categorias) and categorias != '':
        todas_categorias.extend([cat.strip() for cat in str(categorias).split(',')])

categorias_count = Counter(todas_categorias)
for categoria, count in categorias_count.most_common(10):
    porcentaje = (count / len(df_yelp_clean)) * 100
    print(f"   {categoria}: {count} ({porcentaje:.1f}%)")


14.1 ANÁLISIS EXPLORATORIO INICIAL
--------------------------------------------------
📊 Resumen del dataset limpio:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id_yelp               200 non-null    object 
 1   alias                 200 non-null    object 
 2   nombre                200 non-null    object 
 3   calificacion          200 non-null    float32
 4   numero_resenas        200 non-null    int32  
 5   precio_nivel          200 non-null    object 
 6   esta_cerrado          200 non-null    bool   
 7   categorias            200 non-null    object 
 8   categorias_lista      200 non-null    object 
 9   direccion_completa    200 non-null    object 
 10  direccion_calle       200 non-null    object 
 11  ciudad                200 non-null    object 
 12  estado                200 non-null    object 
 13  codigo_p

In [140]:
# =============================================================================
# 14.2 GUARDADO DE DATOS PROCESADOS
# =============================================================================
print("\n14.2 GUARDADO DE DATOS PROCESADOS")
print("-" * 50)

# Guardar datasets para uso futuro
timestamp = time.strftime("%Y%m%d_%H%M%S")
archivo_original = f"yelp_miami_original_{timestamp}.csv"
archivo_limpio = f"yelp_miami_limpio_{timestamp}.csv"

df_yelp.to_csv(archivo_original, index=False, encoding='utf-8')
df_yelp_clean.to_csv(archivo_limpio, index=False, encoding='utf-8')

print(f"✅ Datos originales guardados: {archivo_original}")
print(f"✅ Datos limpios guardados: {archivo_limpio}")
print(f"📦 Tamaño original: {os.path.getsize(archivo_original) / 1024:.1f} KB")
print(f"📦 Tamaño limpio: {os.path.getsize(archivo_limpio) / 1024:.1f} KB")

print("\n" + "=" * 70)
print("✅ EXTRACCIÓN Y TRANSFORMACIÓN COMPLETADA")
print("=" * 70)


14.2 GUARDADO DE DATOS PROCESADOS
--------------------------------------------------
✅ Datos originales guardados: yelp_miami_original_20250828_124858.csv
✅ Datos limpios guardados: yelp_miami_limpio_20250828_124858.csv
📦 Tamaño original: 102.7 KB
📦 Tamaño limpio: 103.3 KB

✅ EXTRACCIÓN Y TRANSFORMACIÓN COMPLETADA


In [141]:
# Variables para uso en siguientes bloques
yelp_data_clean = df_yelp_clean
yelp_data_raw = df_yelp

print("🚀 Datos listos para análisis e integración")

🚀 Datos listos para análisis e integración


Visualizacion del Dataset:

In [142]:
data.keys()
data['businesses']
df_yelp = pd.json_normalize(data['businesses'])
df_yelp

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,...,coordinates.longitude,location.address1,location.address2,location.address3,location.city,location.zip_code,location.country,location.state,location.display_address,price
0,K3ukx2e11xTRtYBU01dmrA,salty-flame-miami,Salty Flame,https://s3-media0.fl.yelpcdn.com/bphoto/_6wShT...,False,https://www.yelp.com/biz/salty-flame-miami?adj...,194,"[{'alias': 'asianfusion', 'title': 'Asian Fusi...",4.4,"[pickup, delivery]",...,-80.192670,1414 Brickell Ave,None,,Miami,33131,US,FL,"[1414 Brickell Ave, Miami, FL 33131]",NaN
1,UXHxLN3DcDGI57uDIfCuJA,olds-havana-cuban-bar-and-cocina-miami,Old's Havana Cuban Bar & Cocina,https://s3-media0.fl.yelpcdn.com/bphoto/OyMD-x...,False,https://www.yelp.com/biz/olds-havana-cuban-bar...,2997,"[{'alias': 'cuban', 'title': 'Cuban'}, {'alias...",4.4,"[restaurant_reservation, pickup]",...,-80.219238,1442 SW 8th St,,None,Miami,33135,US,FL,"[1442 SW 8th St, Miami, FL 33135]",$$
2,oxtMfBGmVNE18pFVuw7lFg,fratellino-coral-gables-2,Fratellino,https://s3-media0.fl.yelpcdn.com/bphoto/OPS9ib...,False,https://www.yelp.com/biz/fratellino-coral-gabl...,1813,"[{'alias': 'italian', 'title': 'Italian'}]",4.8,"[restaurant_reservation, pickup, delivery]",...,-80.260110,264 Miracle Mile,,,Coral Gables,33134,US,FL,"[264 Miracle Mile, Coral Gables, FL 33134]",$$$
3,ix8ifP1jQM9ektdVAs19sQ,crazy-about-you-miami-3,Crazy About You,https://s3-media0.fl.yelpcdn.com/bphoto/vvQBoP...,False,https://www.yelp.com/biz/crazy-about-you-miami...,2787,"[{'alias': 'mediterranean', 'title': 'Mediterr...",4.2,"[restaurant_reservation, delivery]",...,-80.188986,1155 Brickell Bay Dr,Ste 101,,Miami,33131,US,FL,"[1155 Brickell Bay Dr, Ste 101, Miami, FL 33131]",$$
4,WqIBKxRUPXH3NqTViAHLxQ,bayshore-club-bar-and-grill-miami,Bayshore Club Bar & Grill,https://s3-media0.fl.yelpcdn.com/bphoto/B7tTbJ...,False,https://www.yelp.com/biz/bayshore-club-bar-and...,947,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...",4.3,[],...,-80.234162,3391 Pan American Dr,None,None,Miami,33133,US,FL,"[3391 Pan American Dr, Miami, FL 33133]",$$$
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,7zi35PQGV0jnoM7NwHWd-w,klaw-miami-miami,Klaw Miami,https://s3-media0.fl.yelpcdn.com/bphoto/xlCI_b...,False,https://www.yelp.com/biz/klaw-miami-miami?adju...,383,"[{'alias': 'steak', 'title': 'Steakhouses'}, {...",4.4,[],...,-80.185894,1737 N Bayshore Dr,None,,Miami,33132,US,FL,"[1737 N Bayshore Dr, Miami, FL 33132]",$$$$
196,snJPBuR3TtYfapCHuZWsOQ,manjay-miami,Manjay,https://s3-media0.fl.yelpcdn.com/bphoto/PnErNl...,False,https://www.yelp.com/biz/manjay-miami?adjust_c...,158,"[{'alias': 'caribbean', 'title': 'Caribbean'}]",4.3,"[delivery, pickup]",...,-80.193440,8300 NE 2nd Ave,,None,Miami,33138,US,FL,"[8300 NE 2nd Ave, Miami, FL 33138]",$$
197,rotNnhymUbXhXz-WRXlXvg,marias-greek-restaurant-miami-3,Maria's Greek Restaurant,https://s3-media0.fl.yelpcdn.com/bphoto/HNZj6K...,False,https://www.yelp.com/biz/marias-greek-restaura...,349,"[{'alias': 'greek', 'title': 'Greek'}, {'alias...",4.1,"[delivery, pickup]",...,-80.233341,2359 Coral Way,None,,Miami,33145,US,FL,"[2359 Coral Way, Miami, FL 33145]",$$
198,d4pfwfJktfuTAQM51oj_ZQ,montys-coconut-grove-miami-5,Monty's Raw Bar,https://s3-media0.fl.yelpcdn.com/bphoto/Ef17lm...,False,https://www.yelp.com/biz/montys-coconut-grove-...,1311,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...",3.3,"[delivery, pickup]",...,-80.232488,2550 S Bayshore Dr,,,Miami,33133,US,FL,"[2550 S Bayshore Dr, Miami, FL 33133]",$$


In [143]:
# GUARDAR DATOS YELP PARA EL AVANCE 3
df_yelp_clean.to_csv('yelp_miami_limpio.csv', index=False)
print("✅ Datos Yelp de Miami guardados para Avance #3")

✅ Datos Yelp de Miami guardados para Avance #3
